In [1]:
import pathlib


path=str(pathlib.Path().resolve())+"/assets/"

In [ ]:
from ultralytics import YOLO

model = YOLO(path+'yolov8n.pt')


In [ ]:
from collections import defaultdict
from copy import deepcopy
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO(path+'yolov8m.pt')

# Open the video file
video_path = path+"Billar.mp4"
cap = cv2.VideoCapture(video_path)


track_history = defaultdict(lambda: [])
frames=[]

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    #frame = cv2.resize(frame, (int(frame.shape[1]/2),int(frame.shape[0]/2)), interpolation= cv2.INTER_LINEAR)
    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame,imgsz=(int(frame.shape[1]/1.25),int(frame.shape[0]/1.25)), persist=True,conf=0.15,tracker=path+"botsort.yaml")
        boxes = results[0].boxes.xywh.cpu()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()


        track_ids = results[0].boxes.id
        if track_ids is not None:
            track_ids=track_ids.int().cpu().tolist()
            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        frames.append(deepcopy(annotated_frame))
        print(frames)
                # Break the loop if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == 27: break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()